# ETL OUALYON

In [12]:
import pandas as pd
import requests as rq
import json
import MapCategory as CategoryMap
import MapFiltreLyon as FiltreLyonMap

In [13]:
dataframe = pd.DataFrame(columns=['Nom', 'Commune', 'Type','Coord_X', 'Coord_Y'])

#### Import de Sport Gouv

In [14]:
response = rq.get("https://equipements.sports.gouv.fr/api/records/1.0/search/?dataset=data-es&q=&refine.code_dept=69&rows=10000")
data = response.json()
for record in data['records']:
    rc = record['fields']
    dataframe.loc[len(dataframe.index)] = [rc['nominstallation'],rc['nom_commune'],rc['typequipement'],rc['coordgpsx'],rc['coordgpsy']]

#### Import de Data Grand Lyon

In [15]:
response = rq.get("https://download.data.grandlyon.com/ws/rdata/urbalyon.recenseqptsport/all.json?maxfeatures=10000&start=1")
data = response.json()
for record in data['values']: 
    dataframe.loc[len(dataframe.index)] = [record['nom'],record['commune'],record['type'],record['lon'],record['lat']]

In [16]:
# dataframe

#### Traitement des données

In [17]:
dataframe = dataframe.drop_duplicates() # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
dataframe

,Nom,Commune,Type,Coord_X,Coord_Y
0,Base de Loisirs,Aigueperse,Court de tennis,4.436730,46.277250
1,Salle des Fetes et Terrain de Jeu,Alix,Terrain de basket-ball,4.650480,45.914510
2,Collège Saint Viateur,Amplepuis,Terrain de basket-ball,4.330500,45.971400
3,Piscine Municipale,Amplepuis,Bassin sportif de natation,4.336410,45.979650
4,Salle de Gymnastique,Amplepuis,Salle de gymnastique sportive,4.312990,45.981010
...,...,...,...,...,...
8632,court de tennis municipaux 3,Givors,Court de tennis,4.770400,45.598930
8633,Terrain de football 11,Vénissieux,Terrain de football,4.896890,45.720730
8634,court de tennis municipaux 4,Givors,Court de tennis,4.770400,45.598930
8635,COURT DE TENNIS 2,Vénissieux,Court de tennis,4.894418,45.720817


In [28]:
# Gestion des points au même endroit TODO
dataframe.sort_values(by=['Coord_X', 'Coord_Y'])

,Nom,Commune,Type,Coord_X,Coord_Y,Category
3862,Terrain Lafontaine,Villeurbanne,Terrain de basket-ball,4.525252,45.453658,BasketBall
3169,Stade des Peupliers,Villeurbanne,Salle d'haltérophilie,4.535801,45.463992,Salle de sport
3144,Eugene Varlin,Villeurbanne,Structure Artificielle d'Escalade,4.540644,45.460844,Escalade
2647,Ecole Primaire Irene Joliot Curie,Lyon 5e Arrondissement,Salle de basket,4.777677,45.753347,BasketBall
6284,salle de basket-ball,Lyon 5e Arrondissement,Salle de basket,4.777677,45.753347,BasketBall
...,...,...,...,...,...,...
5402,Salle de cardio musculation,Bron,Salle de musculation/cardiotraining,4.931818,45.743478,Salle de sport
6505,Salle de cours collectifs 1,Bron,Salle de cours collectifs,4.931818,45.743478,Salle de sport
7145,Salle de cours collectifs 2,Bron,Salle de cours collectifs,4.931818,45.743478,Salle de sport
7531,Salle de musculation 2,Bron,Salle de musculation/cardiotraining,4.931818,45.743478,Salle de sport


Filtre sur Lyon

In [19]:
# Ajout d'une colonne "Commune = Lyon" avec des valeurs booléens  : True si la valeur de Commune est Lyon ef Nan sinon
FiltreLyonmap = FiltreLyonMap.filtreLyon
dataframe["IsLyon"] = dataframe["Commune"].map(FiltreLyonmap)
# On supprime la colonne qui a été ajouté "Commune=Lyon"
dataframe = dataframe[dataframe["IsLyon"] == True].copy()
dataframe = dataframe.drop(columns='IsLyon')
# dataframe

,Nom,Commune,Type,Coord_X,Coord_Y
52,Parc de Parilly,Bron,Stade d’athlétisme,4.894720,45.717990
53,Parc de Parilly,Bron,Terrain de football,4.897040,45.720450
54,Parc de Parilly,Bron,Terrain de basket-ball,4.895700,45.721680
57,7 Fitness,Bron,Salle de cours collectifs,4.931818,45.743478
58,Stade Léo Lagrange,Bron,Terrain de football,4.925910,45.742140
...,...,...,...,...,...
8616,Palteaux EPS 2,Bron,Plateau EPS/Multisports/city-stades,4.896690,45.721740
8620,Courts extérieurs en green set 2,Villeurbanne,Court de tennis,4.860310,45.781230
8621,Palteaux EPS 3,Bron,Plateau EPS/Multisports/city-stades,4.896690,45.721740
8624,Courts extérieurs en green set 3,Villeurbanne,Court de tennis,4.860310,45.781230


Catégorisation des sports

In [20]:
category_map = CategoryMap.categories

dataframe['Category'] = dataframe['Type'].map(category_map)

In [21]:
# dataframe["Commune"].unique()

In [22]:
dataframe

,Nom,Commune,Type,Coord_X,Coord_Y,Category
52,Parc de Parilly,Bron,Stade d’athlétisme,4.894720,45.717990,Athlétisme
53,Parc de Parilly,Bron,Terrain de football,4.897040,45.720450,FootBall
54,Parc de Parilly,Bron,Terrain de basket-ball,4.895700,45.721680,BasketBall
57,7 Fitness,Bron,Salle de cours collectifs,4.931818,45.743478,Salle de sport
58,Stade Léo Lagrange,Bron,Terrain de football,4.925910,45.742140,FootBall
...,...,...,...,...,...,...
8616,Palteaux EPS 2,Bron,Plateau EPS/Multisports/city-stades,4.896690,45.721740,Autres
8620,Courts extérieurs en green set 2,Villeurbanne,Court de tennis,4.860310,45.781230,Tennis
8621,Palteaux EPS 3,Bron,Plateau EPS/Multisports/city-stades,4.896690,45.721740,Autres
8624,Courts extérieurs en green set 3,Villeurbanne,Court de tennis,4.860310,45.781230,Tennis


#### Export en csv

In [23]:
dataframe.to_csv('activities.csv',sep=";")

#### Export en GeoJSON

In [24]:
# Dico python vide :
GeoJson = {
  "type": "FeatureCollection",
  "features": []
}

In [25]:
# Alimentation des features
for index, row in dataframe.iterrows():
  feature = {
    "type": "Feature",
    "properties": {
      "Name": row["Nom"],
      "Commune" : row["Commune"],
      "Type" : row["Type"],
      "Category" : row["Category"]
    },
    "geometry": {
      "coordinates": [
        row["Coord_X"],
        row["Coord_Y"]
      ],
      "type": "Point"
    }
  }
  
  GeoJson["features"].append(feature)


In [26]:
# convertion en JSON:
GeoJson = json.dumps(GeoJson)

In [27]:
# Création du fichier "GeoActivities.json"
with open("GeoActivities.json", "w") as outfile:
    outfile.write(GeoJson)